In [1]:
import remi.gui as gui
from remi import start, App
from threading import Timer, Thread
import re

In [2]:
remiport = 8085

In [3]:
class MyApp(App):
    def __init__(self, *args):
        super(MyApp, self).__init__(*args)
    
    def _net_interface_ip(self):
        ip = super()._net_interface_ip()
        return ip + f"/proxy/{remiport}"
    
    def _overload(self, data, **kwargs):
        if "filename" in kwargs:
            filename = kwargs['filename']
        else:
            return data
        paths = self.all_paths()
        for pattern in paths.keys():
            if ( filename.endswith(".css") or filename.endswith(".html") or filename.endswith(".js") or filename.endswith("internal") ):
                if type(data) == str:
                    data = re.sub(f"/{pattern}:", f"/proxy/{remiport}/{pattern}:", data)
                else:
                    data = re.sub(f"/{pattern}:", f"/proxy/{remiport}/{pattern}:", data.decode()).encode()
        return data

    def _process_all(self, func, **kwargs):
        print(kwargs)
        kwargs.update({"overload": self._overload})
        super()._process_all(func, **kwargs)

    def main(self):
        #creating a container VBox type, vertical
        wid = gui.VBox(width=300, height=200)

        #creating a text label
        self.lbl = gui.Label('Hello', width='80%', height='50%')

        #a button for simple interaction
        bt = gui.Button('Press me!', width=200, height=30)

        #setting up the listener for the click event
        bt.onclick.connect(self.on_button_pressed)
        
        #adding the widgets to the main container
        wid.append(self.lbl)
        wid.append(bt)

        # returning the root widget
        return wid

    # listener function
    def on_button_pressed(self, emitter):
        # print("button pressed")
        self.lbl.set_text('Hello World!')

In [4]:
myRemi = Thread(target=start, 
                         args=(MyApp,),
                         kwargs={'address':'127.0.0.1', 
                                 'port':remiport, 
                                 'multiple_instance':True,
                                 'enable_file_cache':True, 
                                 'update_interval':0.5, 
                                 'start_browser':False})
myRemi.start()  

remi.server      INFO     Started httpserver http://127.0.0.1:8085/
remi.request     INFO     built UI (path=/)
127.0.0.1 - - [23/Nov/2021 15:54:04] "GET / HTTP/1.1" 200 -


{}


remi.server.ws   INFO     connection established: ('127.0.0.1', 49190)
remi.server.ws   INFO     handshake complete


http://127.0.0.1:8888/proxy/8085/